# 🌍 ERA5 Data Visualization - April 2024
## Domain: 10-40°N, 30-60°E

Interactive visualization of ERA5 hourly data including:
- Surface geopotential & land-sea mask
- 10m and 200hPa winds
- 2m temperature and dewpoint
- Hourly precipitation
- Cloud properties

## 📦 Setup and Imports

In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import ipywidgets as widgets
from IPython.display import display, Markdown

# Set plotting style
plt.rcParams.update({"figure.dpi": 100, "font.size": 10})

print("✅ Imports complete")

## 📂 Load Data

In [ ]:
# ⚠️ IMPORTANT: Update this path to where your ERA5 files are located
# Options:
# 1. If files are uploaded to Claude: "/mnt/user-data/uploads"
# 2. If files are in workspace: "/workspace" or your custom path
# 3. If files are elsewhere: specify the full path

data_dir = "/workspace"  # ⬅️ CHANGE THIS TO YOUR DATA DIRECTORY

# File paths
files = {
    'invariants': os.path.join(data_dir, "ERA-5_10-40N_30-60E_INVARIANTS.nc"),
    'uv10': os.path.join(data_dir, "ERA-5_UV10_10-40N_30-60E_April2024.nc"),
    'temp': os.path.join(data_dir, "ERA-5_TEMP_10-40N_30-60E_April2024.nc"),
    'precip': os.path.join(data_dir, "ERA-5_PRECIP_10-40N_30-60E_April2024.nc"),
    'cloud': os.path.join(data_dir, "ERA-5_CLOUD_10-40N_30-60E_April2024.nc"),
    'uv200': os.path.join(data_dir, "ERA-5_UV200_10-40N_30-60E_April2024.nc")
}

# Load datasets
print("📂 Loading ERA5 datasets...")
datasets = {}
for key, filepath in files.items():
    if os.path.exists(filepath):
        datasets[key] = xr.open_dataset(filepath)
        print(f"✓ Loaded {key}: {os.path.basename(filepath)}")
        # Print coordinate info for debugging
        print(f"  Coords: {list(datasets[key].coords)}")
    else:
        print(f"✗ Missing {key}: {filepath}")

# Display summary
print(f"\n📊 Loaded {len(datasets)} datasets")

## 🗺️ Variable Definitions

In [ ]:
variables = {
    # Invariants
    'var129': {'name': 'Surface Geopotential', 'units': 'm²/s²', 'file': 'invariants', 'cmap': 'terrain'},
    'var172': {'name': 'Land-Sea Mask', 'units': '0-1', 'file': 'invariants', 'cmap': 'RdBu_r'},
    
    # 10m winds
    'var165': {'name': '10m U-wind', 'units': 'm/s', 'file': 'uv10', 'cmap': 'RdBu_r'},
    'var166': {'name': '10m V-wind', 'units': 'm/s', 'file': 'uv10', 'cmap': 'RdBu_r'},
    
    # Temperature
    'var167': {'name': '2m Temperature', 'units': 'K', 'file': 'temp', 'cmap': 'RdYlBu_r'},
    'var168': {'name': '2m Dewpoint', 'units': 'K', 'file': 'temp', 'cmap': 'RdYlBu_r'},
    
    # Precipitation
    'var228': {'name': 'Hourly Precipitation', 'units': 'm', 'file': 'precip', 'cmap': 'YlGnBu'},
    
    # Cloud
    'var23': {'name': 'Cloud Base Height', 'units': 'm', 'file': 'cloud', 'cmap': 'viridis'},
    'var188': {'name': 'High Cloud Cover', 'units': '0-1', 'file': 'cloud', 'cmap': 'gray_r'},
    'var186': {'name': 'Low Cloud Cover', 'units': '0-1', 'file': 'cloud', 'cmap': 'gray_r'},
    'var187': {'name': 'Medium Cloud Cover', 'units': '0-1', 'file': 'cloud', 'cmap': 'gray_r'},
    'var164': {'name': 'Total Cloud Cover', 'units': '0-1', 'file': 'cloud', 'cmap': 'gray_r'},
    
    # 200hPa winds
    'var131': {'name': '200hPa U-wind', 'units': 'm/s', 'file': 'uv200', 'cmap': 'RdBu_r'},
    'var132': {'name': '200hPa V-wind', 'units': 'm/s', 'file': 'uv200', 'cmap': 'RdBu_r'},
}

print(f"📋 Defined {len(variables)} variables for visualization")

## 🔧 Helper Functions

In [ ]:
def get_coords(ds):
    """Get latitude and longitude coordinates handling different naming conventions"""
    # Check for different coordinate names
    lat_names = ['latitude', 'lat', 'Latitude', 'LAT']
    lon_names = ['longitude', 'lon', 'Longitude', 'LON']
    
    lat_coord = None
    lon_coord = None
    
    for name in lat_names:
        if name in ds.coords or name in ds.dims:
            lat_coord = name
            break
    
    for name in lon_names:
        if name in ds.coords or name in ds.dims:
            lon_coord = name
            break
    
    if lat_coord is None or lon_coord is None:
        raise ValueError(f"Could not find lat/lon coordinates. Available coords: {list(ds.coords)}")
    
    return lat_coord, lon_coord

print("✅ Helper functions defined")

## 🎨 Plotting Functions

In [ ]:
def plot_variable(var_name, time_index=0, add_vectors=False):
    """
    Plot a single variable on a map
    
    Parameters:
    -----------
    var_name : str
        Variable code (e.g., 'var167')
    time_index : int
        Time index to plot (for time-varying fields)
    add_vectors : bool
        Add wind vectors (for wind components)
    """
    
    if var_name not in variables:
        print(f"❌ Variable {var_name} not found")
        return
    
    var_info = variables[var_name]
    file_key = var_info['file']
    
    if file_key not in datasets:
        print(f"❌ Dataset {file_key} not loaded")
        return
    
    ds = datasets[file_key]
    
    if var_name not in ds:
        print(f"❌ Variable {var_name} not in dataset")
        return
    
    data = ds[var_name]
    
    # Get coordinate names
    try:
        lat_coord, lon_coord = get_coords(ds)
    except ValueError as e:
        print(f"❌ {e}")
        return
    
    # Select time if available
    if 'time' in data.dims:
        if time_index >= len(data.time):
            time_index = 0
        data = data.isel(time=time_index)
        time_str = f" | {str(data.time.values)[:16]}"
    else:
        time_str = " | Time-invariant"
    
    # Get coordinates
    lat = ds[lat_coord].values
    lon = ds[lon_coord].values
    
    # Create figure with map projection
    fig = plt.figure(figsize=(12, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    # Set extent
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
    
    # Add map features
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
    ax.add_feature(cfeature.LAND, alpha=0.1)
    ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    
    # Plot data
    im = ax.pcolormesh(lon, lat, data.values, 
                       cmap=var_info['cmap'], 
                       transform=ccrs.PlateCarree(),
                       shading='auto')
    
    # Add vectors for wind components
    if add_vectors and file_key in ['uv10', 'uv200']:
        if file_key == 'uv10':
            u = ds['var165'].isel(time=time_index).values if 'time' in ds['var165'].dims else ds['var165'].values
            v = ds['var166'].isel(time=time_index).values if 'time' in ds['var166'].dims else ds['var166'].values
        else:  # uv200
            u = ds['var131'].isel(time=time_index).values if 'time' in ds['var131'].dims else ds['var131'].values
            v = ds['var132'].isel(time=time_index).values if 'time' in ds['var132'].dims else ds['var132'].values
        
        # Subsample for cleaner vectors
        skip = 3
        ax.quiver(lon[::skip], lat[::skip], 
                 u[::skip, ::skip], v[::skip, ::skip],
                 transform=ccrs.PlateCarree(),
                 scale=200, alpha=0.6)
    
    # Colorbar
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
    cbar.set_label(f"{var_info['units']}")
    
    # Title
    title = f"{var_info['name']} ({var_name}){time_str}"
    ax.set_title(title, fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

print("✅ Plotting functions defined")

## 🗺️ Plot Invariant Fields (Time-independent)

In [ ]:
# Plot surface geopotential and land-sea mask
if 'invariants' in datasets:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5), 
                             subplot_kw={'projection': ccrs.PlateCarree()})
    
    ds = datasets['invariants']
    lat_coord, lon_coord = get_coords(ds)
    lat = ds[lat_coord].values
    lon = ds[lon_coord].values
    
    for idx, (var, ax) in enumerate(zip(['var129', 'var172'], axes)):
        var_info = variables[var]
        data = ds[var]
        
        ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
        ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
        
        im = ax.pcolormesh(lon, lat, data.values, 
                          cmap=var_info['cmap'],
                          transform=ccrs.PlateCarree(),
                          shading='auto')
        
        plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
        ax.set_title(f"{var_info['name']}", fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Invariants dataset not loaded")

## 🌡️ Temperature and Humidity Analysis

In [ ]:
# Plot temperature, dewpoint, and calculated relative humidity
time_index = 0  # Change this to select different times

if 'temp' in datasets:
    ds = datasets['temp']
    lat_coord, lon_coord = get_coords(ds)
    
    t2m = ds['var167'].isel(time=time_index)
    td2m = ds['var168'].isel(time=time_index)
    
    # Convert to Celsius
    t2m_c = t2m - 273.15
    td2m_c = td2m - 273.15
    
    # Calculate relative humidity
    rh = 100 * np.exp((17.625 * td2m_c) / (243.04 + td2m_c)) / np.exp((17.625 * t2m_c) / (243.04 + t2m_c))
    
    lat = ds[lat_coord].values
    lon = ds[lon_coord].values
    
    fig = plt.figure(figsize=(16, 5))
    
    for idx, (data, title, cmap) in enumerate([
        (t2m_c, '2m Temperature (°C)', 'RdYlBu_r'),
        (td2m_c, '2m Dewpoint (°C)', 'RdYlBu_r'),
        (rh, 'Relative Humidity (%)', 'BrBG')
    ]):
        ax = plt.subplot(1, 3, idx+1, projection=ccrs.PlateCarree())
        ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
        ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
        
        im = ax.pcolormesh(lon, lat, data.values, 
                          cmap=cmap,
                          transform=ccrs.PlateCarree(),
                          shading='auto')
        plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
        ax.set_title(title, fontweight='bold')
    
    time_str = str(t2m.time.values)[:16]
    fig.suptitle(f'Temperature Analysis | {time_str}', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Temperature dataset not loaded")

## 💨 Wind Analysis (10m)

In [ ]:
# Plot 10m wind speed and vectors
time_index = 0  # Change this

if 'uv10' in datasets:
    ds = datasets['uv10']
    lat_coord, lon_coord = get_coords(ds)
    
    u = ds['var165'].isel(time=time_index)
    v = ds['var166'].isel(time=time_index)
    speed = np.sqrt(u**2 + v**2)
    
    lat = ds[lat_coord].values
    lon = ds[lon_coord].values
    
    fig = plt.figure(figsize=(14, 6))
    
    # Wind speed
    ax1 = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
    ax1.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
    ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax1.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    
    im1 = ax1.pcolormesh(lon, lat, speed.values, 
                        cmap='plasma',
                        transform=ccrs.PlateCarree(),
                        shading='auto')
    plt.colorbar(im1, ax=ax1, label='Wind Speed (m/s)', orientation='horizontal', pad=0.05)
    ax1.set_title('10m Wind Speed', fontweight='bold')
    
    # Wind vectors
    ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
    ax2.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
    ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax2.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
    ax2.add_feature(cfeature.LAND, alpha=0.2)
    ax2.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    
    skip = 2
    ax2.quiver(lon[::skip], lat[::skip], 
              u.values[::skip, ::skip], v.values[::skip, ::skip],
              speed.values[::skip, ::skip],
              transform=ccrs.PlateCarree(),
              cmap='plasma', scale=200, alpha=0.8)
    ax2.set_title('10m Wind Vectors', fontweight='bold')
    
    time_str = str(u.time.values)[:16]
    fig.suptitle(f'10m Wind Analysis | {time_str}', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 10m wind dataset not loaded")

## 💨 Wind Analysis (200hPa)

In [ ]:
# Plot 200hPa wind speed and vectors
time_index = 0  # Change this

if 'uv200' in datasets:
    ds = datasets['uv200']
    lat_coord, lon_coord = get_coords(ds)
    
    u = ds['var131'].isel(time=time_index)
    v = ds['var132'].isel(time=time_index)
    speed = np.sqrt(u**2 + v**2)
    
    lat = ds[lat_coord].values
    lon = ds[lon_coord].values
    
    fig = plt.figure(figsize=(14, 6))
    
    # Wind speed
    ax1 = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
    ax1.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
    ax1.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax1.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
    ax1.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    
    im1 = ax1.pcolormesh(lon, lat, speed.values, 
                        cmap='plasma',
                        transform=ccrs.PlateCarree(),
                        shading='auto')
    plt.colorbar(im1, ax=ax1, label='Wind Speed (m/s)', orientation='horizontal', pad=0.05)
    ax1.set_title('200hPa Wind Speed', fontweight='bold')
    
    # Wind vectors
    ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
    ax2.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
    ax2.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax2.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
    ax2.add_feature(cfeature.LAND, alpha=0.2)
    ax2.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    
    skip = 2
    ax2.quiver(lon[::skip], lat[::skip], 
              u.values[::skip, ::skip], v.values[::skip, ::skip],
              speed.values[::skip, ::skip],
              transform=ccrs.PlateCarree(),
              cmap='plasma', scale=500, alpha=0.8)
    ax2.set_title('200hPa Wind Vectors', fontweight='bold')
    
    time_str = str(u.time.values)[:16]
    fig.suptitle(f'200hPa Wind Analysis | {time_str}', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ 200hPa wind dataset not loaded")

## ⛈️ Precipitation

In [ ]:
# Plot precipitation
time_index = 0  # Change this

plot_variable('var228', time_index=time_index)

## ☁️ Cloud Cover

In [ ]:
# Plot different cloud levels
time_index = 0  # Change this

if 'cloud' in datasets:
    fig, axes = plt.subplots(2, 2, figsize=(14, 12), 
                             subplot_kw={'projection': ccrs.PlateCarree()})
    
    ds = datasets['cloud']
    lat_coord, lon_coord = get_coords(ds)
    lat = ds[lat_coord].values
    lon = ds[lon_coord].values
    
    cloud_vars = ['var186', 'var187', 'var188', 'var164']  # low, mid, high, total
    
    for ax, var in zip(axes.flat, cloud_vars):
        var_info = variables[var]
        data = ds[var].isel(time=time_index)
        
        ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
        ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
        ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
        ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
        
        im = ax.pcolormesh(lon, lat, data.values, 
                          cmap=var_info['cmap'],
                          transform=ccrs.PlateCarree(),
                          vmin=0, vmax=1,
                          shading='auto')
        
        plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
        ax.set_title(f"{var_info['name']}", fontweight='bold')
    
    time_str = str(data.time.values)[:16]
    fig.suptitle(f'Cloud Cover Analysis | {time_str}', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ Cloud dataset not loaded")

## 🎛️ Interactive Variable Explorer

In [ ]:
# Interactive widget for exploring any variable
var_dropdown = widgets.Dropdown(
    options=[(f"{v['name']} ({k})", k) for k, v in variables.items()],
    description='Variable:',
    style={'description_width': '100px'}
)

time_slider = widgets.IntSlider(
    value=0, min=0, max=100, step=1,
    description='Time Index:',
    style={'description_width': '100px'}
)

vectors_check = widgets.Checkbox(
    value=False,
    description='Add wind vectors',
    style={'description_width': '150px'}
)

plot_button = widgets.Button(
    description='Plot Variable',
    button_style='primary'
)

def on_plot_button_click(b):
    plot_variable(var_dropdown.value, time_slider.value, vectors_check.value)

plot_button.on_click(on_plot_button_click)

display(widgets.VBox([var_dropdown, time_slider, vectors_check, plot_button]))

## 📈 Time Series at a Point

In [ ]:
def plot_time_series(var_name, lat_point, lon_point):
    """Plot time series at a specific location"""
    if var_name not in variables:
        print(f"❌ Variable {var_name} not found")
        return
    
    var_info = variables[var_name]
    file_key = var_info['file']
    
    if file_key not in datasets:
        print(f"❌ Dataset not loaded")
        return
    
    ds = datasets[file_key]
    data = ds[var_name]
    
    if 'time' not in data.dims:
        print(f"❌ Variable {var_name} has no time dimension")
        return
    
    # Get coordinate names
    lat_coord, lon_coord = get_coords(ds)
    
    # Select nearest point
    point_data = data.sel({lat_coord: lat_point, lon_coord: lon_point}, method='nearest')
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 4))
    point_data.plot(ax=ax, linewidth=1)
    ax.set_title(f"{var_info['name']} | Lat: {lat_point}°N, Lon: {lon_point}°E")
    ax.set_ylabel(var_info['units'])
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Example: Plot time series of temperature at a point
plot_time_series('var167', lat_point=25.0, lon_point=45.0)

## 📊 Summary Statistics

In [ ]:
# Print summary statistics for each dataset
for key, ds in datasets.items():
    print(f"\n{'='*60}")
    print(f"Dataset: {key}")
    print(f"{'='*60}")
    print(ds)